In [1]:
import numpy as np
import pandas as pd
import warnings
from keras.utils import to_categorical
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
import os
os.getcwd()

'C:\\Users\\p.schambach\\Desktop\\DSR\\drone_steering\\models\\playground'

In [3]:
data_dir = '../../data/gesture/'
os.listdir(data_dir)

['features_flip_c_01_120.csv',
 'features_flip_c_02_120.csv',
 'features_flip_c_03_120.csv',
 'features_flip_l_01_120.csv',
 'features_flip_l_02_120.csv',
 'features_flip_l_03_120.csv',
 'features_flip_p_01_120.csv',
 'features_land_c_01_120.csv',
 'features_land_c_02_120.csv',
 'features_land_c_03_120.csv',
 'features_land_l_01_120.csv',
 'features_land_l_02_120.csv',
 'features_land_l_03_120.csv',
 'features_move_c_01_120.csv',
 'features_move_l_01_120.csv',
 'features_move_p_01_120.csv',
 'features_none_p_01_120.csv',
 'features_takeoff_c_01_120.csv',
 'features_takeoff_c_02_120.csv',
 'features_takeoff_c_03_120.csv',
 'features_takeoff_l_01_120.csv',
 'features_takeoff_l_02_120.csv',
 'features_takeoff_l_03_120.csv',
 'labels_flip_c_01.csv',
 'labels_flip_c_02.csv',
 'labels_flip_c_03.csv',
 'labels_flip_l_01.csv',
 'labels_flip_l_02.csv',
 'labels_flip_l_03.csv',
 'labels_flip_p_01.csv',
 'labels_land_c_01.csv',
 'labels_land_c_02.csv',
 'labels_land_c_03.csv',
 'labels_land_l_01.

In [4]:
import sys
sys.path.insert(0, '../../')

In [5]:
from app_local.module import DataEnsembler, LabelGenerator, GestureTransformer
de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir)
de.load_data()
de.assemble_data(max_error=500, tolerance_range= 600)
de.display_information()

i: 0 	shape X: (537, 17, 16) 	shape y: (537,) 	count: 35
i: 1 	shape X: (520, 17, 16) 	shape y: (520,) 	count: 71
i: 2 	shape X: (499, 17, 16) 	shape y: (499,) 	count: 70
i: 3 	shape X: (557, 17, 16) 	shape y: (557,) 	count: 111
i: 4 	shape X: (509, 17, 16) 	shape y: (509,) 	count: 66
i: 5 	shape X: (467, 17, 16) 	shape y: (467,) 	count: 65
i: 6 	shape X: (532, 17, 16) 	shape y: (532,) 	count: 67
i: 7 	shape X: (441, 17, 16) 	shape y: (441,) 	count: 20
i: 8 	shape X: (491, 17, 16) 	shape y: (491,) 	count: 51
i: 9 	shape X: (477, 17, 16) 	shape y: (477,) 	count: 55
i: 10 	shape X: (493, 17, 16) 	shape y: (493,) 	count: 42
i: 11 	shape X: (437, 17, 16) 	shape y: (437,) 	count: 36
i: 12 	shape X: (467, 17, 16) 	shape y: (467,) 	count: 50
i: 13 	shape X: (649, 17, 16) 	shape y: (649,) 	count: 85
i: 14 	shape X: (588, 17, 16) 	shape y: (588,) 	count: 60
i: 15 	shape X: (565, 17, 16) 	shape y: (565,) 	count: 86
i: 16 	shape X: (541, 17, 16) 	shape y: (541,) 	count: 0
i: 17 	shape X: (470, 17

In [60]:
from keras.utils import to_categorical

X = de.X.copy()
y = de.y.copy()

idx = []
for i in range(7):
    idx.append(np.where(np.isclose(y,i))[0])
    

print(X.shape, y.shape)
print(de.LabelGenerators[0].feature_names)

gt = GestureTransformer(feature_names = list(de.LabelGenerators[0].feature_names),byrow=True)
X = gt.transform(X)
print(X.shape)

y = to_categorical(y)
print(y.shape)

(11723, 17, 16) (11723,)
Index(['leftElbow_x', 'leftElbow_y', 'leftHip_x', 'leftHip_y',
       'leftShoulder_x', 'leftShoulder_y', 'leftWrist_x', 'leftWrist_y',
       'rightElbow_x', 'rightElbow_y', 'rightHip_x', 'rightHip_y',
       'rightShoulder_x', 'rightShoulder_y', 'rightWrist_x', 'rightWrist_y'],
      dtype='object')
(11723, 17, 16)
(11723, 7)


In [61]:
for i in range(7):
    print(i,len(idx[i]))
    


0 10556
1 197
2 254
3 231
4 0
5 0
6 485


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

In [28]:
n_timesteps = X.shape[1] # here 17
n_features = X.shape[2] # here 16
n_outputs =  y.shape[1] # here 7 (number of labels)

model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.4))
model.add(Dense(100, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               46800     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 455       
Total params: 63,819
Trainable params: 63,819
Non-trainable params: 0
_________________________________________________________________


In [30]:
history = model.fit(        
    X, 
    y, 
    epochs=50, 
    batch_size=batch_size, 
    verbose=1
)

Epoch 1/50
1167/1167 [==============================] - 0s 156us/step - loss: 0.1301 - acc: 0.9452
Epoch 2/50
1167/1167 [==============================] - 0s 147us/step - loss: 0.1049 - acc: 0.9589
Epoch 3/50
1167/1167 [==============================] - 0s 142us/step - loss: 0.0998 - acc: 0.9632
Epoch 4/50
1167/1167 [==============================] - 0s 144us/step - loss: 0.0918 - acc: 0.9589
Epoch 5/50
1167/1167 [==============================] - 0s 158us/step - loss: 0.1047 - acc: 0.9554
Epoch 6/50
1167/1167 [==============================] - 0s 149us/step - loss: 0.1155 - acc: 0.9554
Epoch 7/50
1167/1167 [==============================] - 0s 151us/step - loss: 0.1142 - acc: 0.9529
Epoch 8/50
1167/1167 [==============================] - 0s 141us/step - loss: 0.1036 - acc: 0.9580
Epoch 9/50
1167/1167 [==============================] - 0s 157us/step - loss: 0.0860 - acc: 0.9657
Epoch 10/50
1167/1167 [==============================] - 0s 150us/step - loss: 0.0828 - acc: 0.9709
Epoch 11/

In [31]:
model.evaluate(
    X,
    y,
    batch_size=128,
    verbose=1
)

1167/1167 [==============================] - 0s 214us/step


[0.07409453733912777, 0.9648671808054842]

In [47]:
model.predict_classes(X[0:1,:,:])[0]
np.nonzero(y[731])[0][0]

2

In [49]:
for i in range(X.shape[0]):
    print("Index:",i,"\tPred:",model.predict_classes(X[i:(i+1),:,:])[0],"\tActual:",np.nonzero(y[i])[0][0])

Index: 0 	Pred: 6 	Actual: 6
Index: 1 	Pred: 6 	Actual: 6
Index: 2 	Pred: 6 	Actual: 6
Index: 3 	Pred: 6 	Actual: 6
Index: 4 	Pred: 6 	Actual: 6
Index: 5 	Pred: 6 	Actual: 6
Index: 6 	Pred: 6 	Actual: 6
Index: 7 	Pred: 6 	Actual: 6
Index: 8 	Pred: 6 	Actual: 6
Index: 9 	Pred: 6 	Actual: 6
Index: 10 	Pred: 6 	Actual: 6
Index: 11 	Pred: 6 	Actual: 6
Index: 12 	Pred: 6 	Actual: 6
Index: 13 	Pred: 6 	Actual: 6
Index: 14 	Pred: 6 	Actual: 6
Index: 15 	Pred: 6 	Actual: 6
Index: 16 	Pred: 6 	Actual: 6
Index: 17 	Pred: 6 	Actual: 6
Index: 18 	Pred: 6 	Actual: 6
Index: 19 	Pred: 1 	Actual: 6
Index: 20 	Pred: 1 	Actual: 6
Index: 21 	Pred: 1 	Actual: 6
Index: 22 	Pred: 1 	Actual: 6
Index: 23 	Pred: 1 	Actual: 6
Index: 24 	Pred: 6 	Actual: 6
Index: 25 	Pred: 6 	Actual: 6
Index: 26 	Pred: 6 	Actual: 6
Index: 27 	Pred: 6 	Actual: 6
Index: 28 	Pred: 6 	Actual: 6
Index: 29 	Pred: 6 	Actual: 6
Index: 30 	Pred: 6 	Actual: 6
Index: 31 	Pred: 6 	Actual: 6
Index: 32 	Pred: 6 	Actual: 6
Index: 33 	Pred: 6 	

Index: 400 	Pred: 6 	Actual: 6
Index: 401 	Pred: 6 	Actual: 6
Index: 402 	Pred: 6 	Actual: 6
Index: 403 	Pred: 6 	Actual: 6
Index: 404 	Pred: 6 	Actual: 6
Index: 405 	Pred: 6 	Actual: 6
Index: 406 	Pred: 6 	Actual: 6
Index: 407 	Pred: 6 	Actual: 6
Index: 408 	Pred: 1 	Actual: 6
Index: 409 	Pred: 1 	Actual: 6
Index: 410 	Pred: 1 	Actual: 6
Index: 411 	Pred: 6 	Actual: 6
Index: 412 	Pred: 6 	Actual: 6
Index: 413 	Pred: 6 	Actual: 6
Index: 414 	Pred: 6 	Actual: 6
Index: 415 	Pred: 1 	Actual: 6
Index: 416 	Pred: 6 	Actual: 6
Index: 417 	Pred: 6 	Actual: 6
Index: 418 	Pred: 6 	Actual: 6
Index: 419 	Pred: 6 	Actual: 6
Index: 420 	Pred: 6 	Actual: 6
Index: 421 	Pred: 6 	Actual: 6
Index: 422 	Pred: 6 	Actual: 6
Index: 423 	Pred: 6 	Actual: 6
Index: 424 	Pred: 6 	Actual: 6
Index: 425 	Pred: 6 	Actual: 6
Index: 426 	Pred: 6 	Actual: 6
Index: 427 	Pred: 6 	Actual: 6
Index: 428 	Pred: 6 	Actual: 6
Index: 429 	Pred: 6 	Actual: 6
Index: 430 	Pred: 6 	Actual: 6
Index: 431 	Pred: 6 	Actual: 6
Index: 4

Index: 799 	Pred: 3 	Actual: 3
Index: 800 	Pred: 3 	Actual: 3
Index: 801 	Pred: 3 	Actual: 3
Index: 802 	Pred: 3 	Actual: 3
Index: 803 	Pred: 3 	Actual: 3
Index: 804 	Pred: 3 	Actual: 3
Index: 805 	Pred: 3 	Actual: 3
Index: 806 	Pred: 3 	Actual: 3
Index: 807 	Pred: 3 	Actual: 3
Index: 808 	Pred: 3 	Actual: 3
Index: 809 	Pred: 3 	Actual: 3
Index: 810 	Pred: 3 	Actual: 3
Index: 811 	Pred: 3 	Actual: 3
Index: 812 	Pred: 3 	Actual: 3
Index: 813 	Pred: 3 	Actual: 3
Index: 814 	Pred: 3 	Actual: 3
Index: 815 	Pred: 3 	Actual: 3
Index: 816 	Pred: 3 	Actual: 3
Index: 817 	Pred: 3 	Actual: 3
Index: 818 	Pred: 3 	Actual: 3
Index: 819 	Pred: 3 	Actual: 3
Index: 820 	Pred: 3 	Actual: 3
Index: 821 	Pred: 3 	Actual: 3
Index: 822 	Pred: 3 	Actual: 3
Index: 823 	Pred: 3 	Actual: 3
Index: 824 	Pred: 3 	Actual: 3
Index: 825 	Pred: 3 	Actual: 3
Index: 826 	Pred: 3 	Actual: 3
Index: 827 	Pred: 3 	Actual: 3
Index: 828 	Pred: 3 	Actual: 3
Index: 829 	Pred: 3 	Actual: 3
Index: 830 	Pred: 3 	Actual: 3
Index: 8